<a href="https://colab.research.google.com/github/lawlete/Varios/blob/main/Entrenamiento_de_Lenguaje_Natural_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ejercicio: Procesamiento de Lenguaje Natural (NLP)**

Utiliza Python y una librería como Hugging Face Transformers para entrenar un modelo que realice una tarea de clasificación de texto (por ejemplo, clasificación de opiniones: positiva o negativa). Proporciona un conjunto de datos de ejemplo (puede ser un pequeño conjunto de datos de reseñas) y demuestra cómo entrenar el modelo, evaluarlo y ajustarlo.

**Respuesta:**
Este script utiliza Python y la librería Hugging Face Transformers para entrenar un modelo de clasificación de texto. En este caso, el modelo se entrena para clasificar reseñas de películas como positivas o negativas utilizando el conjunto de datos IMDB.

Además tambien, se integra Weights & Biases (W&B) para registrar métricas, hiperparámetros y artefactos durante el entrenamiento y la evaluación del modelo.

**Breve comentarios sobre Weights & Biases (W&B)**

Weights & Biases (W&B) es una plataforma de experimentación para machine learning que permite registrar, visualizar y comparar experimentos. Al integrar W&B en este script, podemos aprovechar varias funciones y ventajas:

1. **Registro de Métricas**: W&B registra automáticamente métricas como la precisión y la pérdida durante el entrenamiento y la evaluación del modelo. Esto permite un seguimiento detallado del rendimiento del modelo a lo largo del tiempo.

2. **Registro de Hiperparámetros**: Los hiperparámetros configurados al iniciar el experimento (como la tasa de aprendizaje, el tamaño del lote y el número de épocas) se registran automáticamente. Esto facilita la reproducción de experimentos y la comparación de diferentes configuraciones.

3. **Visualización de Resultados**: W&B proporciona gráficos y paneles interactivos para visualizar el rendimiento del modelo. Esto incluye gráficos de pérdida y precisión, así como visualizaciones de hiperparámetros.

4. **Comparación de Experimentos**: W&B permite comparar fácilmente diferentes experimentos y configuraciones de modelos. Esto es útil para identificar qué configuraciones de hiperparámetros o qué modelos preentrenados funcionan mejor para la tarea específica.

5. **Colaboración**: W&B facilita el trabajo en equipo al permitir compartir experimentos y resultados con otros colaboradores. Esto es especialmente útil en proyectos de investigación o desarrollo colaborativo.

6. **Registro de Artefactos**: Los modelos guardados durante el entrenamiento se registran como artefactos en W&B. Esto permite un seguimiento claro de las versiones del modelo y facilita la gestión de modelos.



Paso 1: Instalación de dependencias
Primero, instala las librerías necesarias

In [1]:
!pip install python-dotenv
!pip install transformers datasets torch
!pip install wandb  # Instalar Weights & Biases


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Cargamos la API TOKEN de Hugging Face**

In [2]:
from dotenv import load_dotenv, find_dotenv
import os
# Verifica y carga el file con las variables de entorno si existe
if load_dotenv(find_dotenv()):
    if "HF_TOKEN" in os.environ: # Verifica que existe la variable HF_KEY
        print("HUGGING FACE API TOKEN cargada") # Imprime el mensaje Si existe la variable de OpenAI
    else:
        print("No se cargo la api_key de HUGGING FACE, debera cargarla manualmente")
else:
      print("No se cargo la api_key, debera cargarla manualmente")

HUGGING FACE API TOKEN cargada


**Inicializamos Weights & Biases (W&B)**

In [31]:
import wandb

# Iniciar un nuevo experimento en W&B
wandb.init(project="text-classification-imdb", config={
    "learning_rate": 2e-5,
    "batch_size": 16,
    "epochs": 3,
    "weight_decay": 0.01,
})

Paso 2: Cargar un conjunto de datos
Utilizaremos el conjunto de datos IMDB, que contiene reseñas de películas etiquetadas como positivas (1) o negativas (0). Este conjunto de datos está disponible en la librería datasets de Hugging Face.

In [4]:
from datasets import load_dataset

# Cargar el conjunto de datos IMDB
dataset = load_dataset("imdb")

# Ver la estructura del dataset
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


**Paso 3: Preprocesamiento de los datos**

**Tokenizamos el texto utilizando un tokenizador preentrenado de Hugging Face.**

**En este caso, usaremos el modelo "distilbert-base-uncased", entre los que hay disponibles en la lista de abajo.**

**Lista de modelos para usar:**

1. **BERT (Bidirectional Encoder Representations from Transformers)**:
   - `bert-base-uncased`: La versión base de BERT sin distinción de mayúsculas y minúsculas.
   - `bert-large-uncased`: Una versión más grande de BERT sin distinción de mayúsculas y minúsculas.
   - `bert-base-cased`: La versión base de BERT con distinción de mayúsculas y minúsculas.
   - `bert-large-cased`: Una versión más grande de BERT con distinción de mayúsculas y minúsculas.

2. **RoBERTa (Robustly Optimized BERT approach)**:
   - `roberta-base`: La versión base de RoBERTa.
   - `roberta-large`: Una versión más grande de RoBERTa.

3. **AlBERT (A Lite BERT)**:
   - `albert-base-v2`: La versión base de AlBERT.
   - `albert-large-v2`: Una versión más grande de AlBERT.

4. **XLNet**:
   - `xlnet-base-cased`: La versión base de XLNet con distinción de mayúsculas y minúsculas.
   - `xlnet-large-cased`: Una versión más grande de XLNet con distinción de mayúsculas y minúsculas.

5. **ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)**:
   - `electra-base-uncased`: La versión base de ELECTRA sin distinción de mayúsculas y minúsculas.
   - `electra-large-uncased`: Una versión más grande de ELECTRA sin distinción de mayúsculas y minúsculas.

6. **DeBERTa (Decoding-enhanced BERT with disentangled attention)**:
   - `deberta-base`: La versión base de DeBERTa.
   - `deberta-large`: Una versión más grande de DeBERTa.

7. **Longformer**:
   - `longformer-base-4096`: La versión base de Longformer, diseñada para manejar secuencias largas.
   - `longformer-large-4096`: Una versión más grande de Longformer.

8. **BigBird**:
   - `bigbird-roberta-base`: La versión base de BigBird, diseñada para manejar secuencias largas.
   - `bigbird-roberta-large`: Una versión más grande de BigBird.

9. **TinyBERT**:
   - `tiny-bert-base-uncased`: Una versión más pequeña y rápida de BERT sin distinción de mayúsculas y minúsculas.

10. **MobileBERT**:
    - `mobilebert-uncased`: Una versión más pequeña y rápida de BERT sin distinción de mayúsculas y minúsculas.



In [5]:
from transformers import AutoTokenizer

#
# Modelos posibles a usar listados arriba
#

modelo = "distilbert-base-uncased"

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(modelo)

# Función para tokenizar el texto
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Aplicar tokenización al dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Paso 4: Dividir el conjunto de datos**

**Dividimos el conjunto de datos en entrenamiento y evaluación.**

In [6]:
# Dividir el dataset en entrenamiento y evaluación
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(100))  # Usamos solo 100 muestras para entrenamiento rápido
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(10))    # Usamos 10 muestras para evaluación

**Paso 5: Entrenar el modelo**

**Cargamos un modelo preentrenado y lo ajustamos para la tarea de clasificación.**

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [29]:
!pip install scikit-learn # Usado para definir las metricas


In [33]:
from sklearn.metrics import accuracy_score

# Definir la métrica de precisión
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}


# Reconfigirar los hiperparámetros en W&B
config = wandb.config
config.learning_rate = 2e-5
config.batch_size = 16
config.epochs = 3
config.weight_decay = 0.01


# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=wandb.config.learning_rate,  # Usar el learning_rate configurado en W&B
    per_device_train_batch_size=wandb.config.batch_size,  # Usar el batch_size configurado en W&B
    per_device_eval_batch_size=wandb.config.batch_size,
    num_train_epochs=wandb.config.epochs,  # Usar el número de épocas configurado en W&B
    weight_decay=wandb.config.weight_decay,  # Usar el weight_decay configurado en W&B
    logging_dir="./logs",  # Directorio para guardar logs
    logging_steps=10,  # Registrar métricas cada 10 pasos
    report_to="wandb",  # Enviar métricas a W&B
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,  # Agregar la función de métricas
)

# Entrenar el modelo
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.661162,0.800000
2,0.590900,0.620539,0.700000
3,0.501400,0.608759,0.700000


TrainOutput(global_step=21, training_loss=0.5361053077947526, metrics={'train_runtime': 21.7707, 'train_samples_per_second': 13.78, 'train_steps_per_second': 0.965, 'total_flos': 39740219596800.0, 'train_loss': 0.5361053077947526, 'epoch': 3.0})

**Paso 6: Evaluar el modelo**

**Evaluamos el modelo en el conjunto de evaluación.**

In [35]:
# Evaluar el modelo
eval_results = trainer.evaluate()
print(f"Precisión: {eval_results['eval_accuracy']}")
print(f"Loss: {eval_results['eval_loss']}")

# Registrar la precisión y la pérdida en W&B
wandb.log({"Precisión": eval_results["eval_accuracy"]})
wandb.log({"Loss": eval_results["eval_loss"]})


Precisión: 0.7
Loss: 0.6087592840194702


**Paso 7: Realizar predicciones**

**Podemos usar el modelo entrenado para clasificar nuevas reseñas.**  

In [36]:
import torch

# Verificar si hay una GPU disponible y mover el modelo al dispositivo adecuado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Función para predecir la clase de un texto
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover los tensores al mismo dispositivo
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)
    return "Positiva" if predictions.item() == 1 else "Negativa"

# Probamos el modelo entrenado con una lista de comentarios

comentarios = [
        "I loved this movie, it was amazing!",
        "Terrible film, I hated it.",
        "The acting was great, but the plot was boring.",
        "Absolutely fantastic!",
        "Worst movie ever.",
        "The movie was horrible, terribly bad.",
        "This movie was fantastic! I loved every minute of it."
    ]

for i, comentario in enumerate(comentarios):
    predict = predict_sentiment(comentario)
    print(f"Comentario {i + 1}:", comentario)
    print(f"Predicción: {predict}", "\n")
    wandb.log({f"Comentario {i + 1}": comentario})
    wandb.log({f"Prediccion {i + 1}": predict})



Comentario 1: I loved this movie, it was amazing!
Predicción: Positiva 

Comentario 2: Terrible film, I hated it.
Predicción: Negativa 

Comentario 3: The acting was great, but the plot was boring.
Predicción: Positiva 

Comentario 4: Absolutely fantastic!
Predicción: Positiva 

Comentario 5: Worst movie ever.
Predicción: Negativa 

Comentario 6: The movie was horrible, terribly bad.
Predicción: Negativa 

Comentario 7: This movie was fantastic! I loved every minute of it.
Predicción: Positiva 



In [37]:
# Finalizar el experimento en W&B e imprimimos los parametros
wandb.finish()

Loss,▁
Precisión,▁
eval/accuracy,█▁▁▁▁
eval/loss,█▆▂▁▁▁
eval/runtime,▅▃▂▆▁█
eval/samples_per_second,▃▅▇▂█▁
eval/steps_per_second,▃▅▇▂█▁
train/epoch,█▁▃▅█████
train/global_step,█▁▃▅▇████████████████████
train/grad_norm,▁█
train/learning_rate,█▁


**Graficos de evolucion del entrenamiento**

https://wandb.ai/alfredolawler-lawer-technology/huggingface/runs/pzhtx365?nw=nwuseralfredolawler


In [24]:
# Reinicio el registro de parametros en W&B
wandb.init()

**Paso 8: Ajustar el modelo (opcional)**

Si el rendimiento no es satisfactorio, es posible ajustar los hiperparámetros (por ejemplo, learning_rate, num_train_epochs, batch_size) o probar con otro modelo preentrenado.